# Factor analysis

In [1]:
import warnings
warnings.filterwarnings('ignore')
from utils import create_dataset_mri, create_dataset_eeg, create_dataset_eeg_old
import numpy as np
import tensorflow as tf
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, balanced_accuracy_score
from sklearn.decomposition import FactorAnalysis

In [36]:
# importing behavioural data
behavioral = pd.read_csv('data/Behavioral/cleaned/HBNFinalSummaries.csv')
behavioral 



,Anonymized.ID,EID,Sex,Age,Study.Site,NoDX,DX_01_Cat,DX_01_Sub,DX_01,DX_01_Spec,...,WISC_PSI,WISC_FSIQ,YSR_AB,YSR_AD,YSR_WD,YSR_RBB,YSR_SC,YSR_Ext,YSR_Int,YSR_Total
1,A00078864,NDARYM832PX3,1,7.048254,1,Yes,Anxiety Disorders,NaN,Generalized Anxiety Disorder,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A00078865,NDARNJ687DMC,1,6.348163,1,Yes,Depressive Disorders,NaN,Disruptive Mood Dysregulation Disorder,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A00078866,NDARRM363BXZ,0,10.052589,1,Yes,Neurodevelopmental Disorders,Attention-Deficit/Hyperactivity Disorder,ADHD-Combined Type,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A00078867,NDARUW586LLL,1,12.319415,1,Yes,Depressive Disorders,NaN,Major Depressive Disorder,Single episode,...,NaN,NaN,9.0,12.0,8.0,7.0,9.0,16.0,29.0,85.0
5,A00078868,NDARDC298NW4,0,13.901437,1,Yes,Neurodevelopmental Disorders,Intellectual Disability,Intellectual Disability-Mild,NaN,...,NaN,NaN,8.0,10.0,5.0,2.0,11.0,10.0,26.0,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,A00093557,NDARYZ986HEW,1,7.563084,3,Yes,Neurodevelopmental Disorders,Attention-Deficit/Hyperactivity Disorder,ADHD-Combined Type,NaN,...,116.0,129.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2093,A00093558,NDARPM572ZZV,0,9.832762,3,Yes,Neurodevelopmental Disorders,Specific Learning Disorder,Specific Learning Disorder with Impairment in ...,rule out,...,98.0,112.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2094,A00093559,NDARYA955CY1,0,15.357859,3,Yes,Neurodevelopmental Disorders,Specific Learning Disorder,Specific Learning Disorder with Impairment in ...,NaN,...,92.0,111.0,5.0,5.0,4.0,6.0,4.0,11.0,13.0,47.0
2095,A00093560,NDARRT283RHB,1,10.712069,1,Dropped out before diagnosis was given,No Diagnosis Given: Incomplete Eval,NaN,No Diagnosis Given: Incomplete Eval,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# pop non useful columns
behavioral.pop('NoDX')
behavioral.pop('Study.Site')
behavioral.pop('Anonymized.ID')

1       A00078864
2       A00078865
3       A00078866
4       A00078867
5       A00078868
          ...    
2092    A00093557
2093    A00093558
2094    A00093559
2095    A00093560
2096    A00093561
Name: Anonymized.ID, Length: 2096, dtype: object

In [38]:
id_column = behavioral.pop('EID')

In [39]:
diagnoses_data = behavioral.loc[:,'DX_01_Cat':'DX_10_PRem']

In [40]:
behavioral = behavioral.drop(columns=behavioral.loc[:,'DX_01_Cat':'DX_10_PRem'].columns)

In [34]:
for row in behavioral.index: 
    if behavioral.loc[row,:].isna().sum() <= 200 : 
        print(row)

6
7
30
32
33
36
37
41
46
65
66
71
75
77
82
87
88
94
96
97
101
109
110
112
113
119
120
126
128
129
130
138
141
144
145
149
152
154
158
162
168
173
174
175
180
183
184
189
192
193
195
197
198
203
205
206
211
214
215
216
219
220
230
231
233
234
235
236
243
244
247
248
250
251
253
254
255
258
264
266
267
268
269
270
271
272
273
276
277
280
281
282
283
284
285
287
289
291
292
293
294
297
298
299
301
302
303
305
306
311
312
313
314
315
316
317
320
321
322
323
324
325
327
328
329
332
333
334
335
336
337
338
340
341
342
343
345
346
347
348
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
368
369
371
372
373
375
377
378
379
380
381
382
383
384
385
389
391
392
395
396
397
398
399
401
402
403
404
405
408
409
410
411
421
423
424
426
427
428
430
432
433
434
435
437
438
439
440
441
442
443
444
445
446
447
448
449
451
452
453
454
455
456
457
459
460
461
463
464
465
466
468
469
470
472
473
475
476
477
478
479
480
481
482
483
484
485
487
488
489
491
492
493
495
496
497
498
500
501
50

In [42]:
# creating feature for nans 
behavioral_copy = behavioral.copy()
for column in behavioral.columns: 
    if behavioral[column].isna().any(): 
        nan_column_name = column+'_nan'
        nan_column = behavioral[column].isna()
        behavioral[nan_column_name] = nan_column.astype('int64')
behavioral

,Sex,Age,ACE_Score,APQ_P_OPD,APQ_P_Total,APQ_SR_OPD,APQ_SR_Total,ARI_P_Total_Score,ARI_S_Total_Score,ASR_Total,...,WISC_PSI_nan,WISC_FSIQ_nan,YSR_AB_nan,YSR_AD_nan,YSR_WD_nan,YSR_RBB_nan,YSR_SC_nan,YSR_Ext_nan,YSR_Int_nan,YSR_Total_nan
1,1,7.048254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1
2,1,6.348163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1
3,0,10.052589,NaN,NaN,NaN,17.0,118.0,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1
4,1,12.319415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,0,0,0,0,0,0,0,0
5,0,13.901437,NaN,NaN,NaN,33.0,154.0,NaN,NaN,NaN,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,1,7.563084,NaN,21.0,96.0,19.0,116.0,10.0,8.0,NaN,...,0,0,1,1,1,1,1,1,1,1
2093,0,9.832762,NaN,20.0,96.0,14.0,115.0,2.0,3.0,NaN,...,0,0,1,1,1,1,1,1,1,1
2094,0,15.357859,NaN,17.0,113.0,15.0,128.0,3.0,1.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2095,1,10.712069,NaN,NaN,NaN,20.0,113.0,NaN,1.0,NaN,...,1,1,1,1,1,1,1,1,1,1


In [43]:
# fill nans
behavioral = behavioral.fillna(0)

In [44]:
behavioral

,Sex,Age,ACE_Score,APQ_P_OPD,APQ_P_Total,APQ_SR_OPD,APQ_SR_Total,ARI_P_Total_Score,ARI_S_Total_Score,ASR_Total,...,WISC_PSI_nan,WISC_FSIQ_nan,YSR_AB_nan,YSR_AD_nan,YSR_WD_nan,YSR_RBB_nan,YSR_SC_nan,YSR_Ext_nan,YSR_Int_nan,YSR_Total_nan
1,1,7.048254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
2,1,6.348163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
3,0,10.052589,0.0,0.0,0.0,17.0,118.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
4,1,12.319415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,0,0,0,0,0,0,0,0
5,0,13.901437,0.0,0.0,0.0,33.0,154.0,0.0,0.0,0.0,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,1,7.563084,0.0,21.0,96.0,19.0,116.0,10.0,8.0,0.0,...,0,0,1,1,1,1,1,1,1,1
2093,0,9.832762,0.0,20.0,96.0,14.0,115.0,2.0,3.0,0.0,...,0,0,1,1,1,1,1,1,1,1
2094,0,15.357859,0.0,17.0,113.0,15.0,128.0,3.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2095,1,10.712069,0.0,0.0,0.0,20.0,113.0,0.0,1.0,0.0,...,1,1,1,1,1,1,1,1,1,1


In [54]:
# convert variable of object type to categorical
for column in behavioral: 
    if behavioral[column].dtypes=='object': 
        behavioral[column]=pd.Categorical(behavioral[column]).astype('int64')


In [60]:
# convert categorical variables to integers
cat_columns = behavioral.select_dtypes(['category']).columns
behavioral[cat_columns] = behavioral[cat_columns]. apply(lambda x: x.cat.codes)

In [62]:
# factor analysis 
fa = FactorAnalysis(n_components=10).fit(behavioral)
factors = pd.DataFrame(fa.components_, index = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10'], columns = behavioral.columns)

In [70]:
for index in factors.index: 
    print('\n\n',index,'\n')
    for column in factors.columns: 
        if factors.loc[index,column] >= 10 or factors.loc[index,column] <= -10: 
            print(column)



 f1 

APQ_P_Total
APQ_SR_Total
CBCL_Total
CTOPP_RSN_Comp
SRS_DSMRRB
SRS_MOT
SRS_SCI
SRS_Total
run1_ed50
run2_ed50
TOWRE_SWE_Raw
WIAT_Word_Raw
WIAT_MP_Raw
WISC_VSI
WISC_VCI
WISC_FRI
WISC_WMI
WISC_PSI
WISC_FSIQ


 f2 

APQ_P_Total
run1_ed50
run2_ed50


 f3 

APQ_P_Total
APQ_SR_Total
CTOPP_RSN_Comp
PSI_Total
run1_ed50
run2_ed50


 f4 

run1_ed50
run2_ed50


 f5 

run1_ed50
run2_ed50


 f6 

fdxsev
mdxsev


 f7 

fdxsev
mdxsev


 f8 

APQ_SR_Total
CTOPP_RSN_Comp
sib1dxse
PSI_Total
SDS_Total_Raw
TOWRE_SWE_Raw
WASI_VCI_Comp
WASI_PRI_Comp
WISC_Coding_Raw
WISC_VSI
WISC_VCI
WISC_FRI
WISC_WMI
WISC_PSI
WISC_FSIQ


 f9 

sib1dxse
sib2dxse


 f10 

CTOPP_RSN_Comp
Processing_Raw
PSI_Total
TOWRE_SWE_Raw
